In [1]:
from src.mazeData import *
from src.maze2025GraphClass import mazeGraph
from pyvis.network import Network
from src.PS_agentPrograms import *
from src.mazeProblemClass import MazeProblem
from src.agents import *
from src.naigationEnvironmentClass import MazeNavigationEnvironment
from src.Lab5Env import *

In [2]:
def makeMaze(n):
  size = (n,n)
  proba_0 =0.20 # resulting array will have 25%? of zeros
  proba_Food =0.1 # resulting array will have 10%? of enemy
  arrMaze=np.random.choice([0, 1, 2], size=size, p=[proba_0, 1-proba_0-proba_Food, proba_Food] )

  return arrMaze

initState = (random.randint(0,9),random.randint(0,9))
goalState = (random.randint(0,9),random.randint(0,9))

Pinky = (random.randint(0,9),random.randint(0,9))
Inky = (random.randint(0,9),random.randint(0,9))
Blinky = (random.randint(0,9),random.randint(0,9))
Clyde = (random.randint(0,9),random.randint(0,9))
Betrayus = (random.randint(0,9),random.randint(0,9))

goals = [goalState]

mazeSize=10
mainMaze = makeMaze(mazeSize)
MazeCheck(mainMaze,initState,goalState)
GhostCheck(mainMaze,Pinky,Inky,Blinky,Clyde,Betrayus)
findPelletGoals(goals,mainMaze)
mazeAvalActs=defineMazeAvailableActions(mainMaze)
maze1TM=makeMazeTransformationModel(mazeAvalActs)
mazeWorldGraph=mazeGraph(maze1TM, mazeStatesLocations(list(maze1TM.keys())))

maze1TM

{(0, 0): {'right': (0, 1), 'down': (1, 0)},
 (0, 1): {'left': (0, 0), 'right': (0, 2)},
 (0, 2): {'left': (0, 1), 'right': (0, 3)},
 (0, 3): {'left': (0, 2), 'down': (1, 3)},
 (0, 4): {},
 (0, 5): {'right': (0, 6), 'down': (1, 5)},
 (0, 6): {'left': (0, 5), 'right': (0, 7), 'down': (1, 6)},
 (0, 7): {'left': (0, 6), 'right': (0, 8)},
 (0, 8): {'left': (0, 7), 'right': (0, 9), 'down': (1, 8)},
 (0, 9): {'left': (0, 8), 'down': (1, 9)},
 (1, 0): {'up': (0, 0), 'down': (2, 0)},
 (1, 1): {},
 (1, 2): {},
 (1, 3): {'up': (0, 3), 'down': (2, 3)},
 (1, 4): {},
 (1, 5): {'up': (0, 5), 'right': (1, 6), 'down': (2, 5)},
 (1, 6): {'left': (1, 5), 'up': (0, 6), 'down': (2, 6)},
 (1, 7): {},
 (1, 8): {'up': (0, 8), 'right': (1, 9)},
 (1, 9): {'left': (1, 8), 'up': (0, 9), 'down': (2, 9)},
 (2, 0): {'up': (1, 0), 'right': (2, 1), 'down': (3, 0)},
 (2, 1): {'left': (2, 0), 'right': (2, 2), 'down': (3, 1)},
 (2, 2): {'left': (2, 1), 'right': (2, 3), 'down': (3, 2)},
 (2, 3): {'left': (2, 2), 'up': (1,

In [3]:
mainMaze

array([[1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
       [2, 0, 0, 1, 0, 3, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 0, 1, 1, 1],
       [3, 0, 0, 2, 2, 2, 1, 0, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 2, 1, 1, 1, 0, 3, 1, 3],
       [0, 1, 1, 1, 0, 0, 1, 1, 0, 3],
       [1, 1, 1, 1, 1, 0, 1, 1, 0, 0]])

In [4]:
goals

[(8, 2), (1, 0), (4, 3), (4, 4), (4, 5), (7, 2)]

In [5]:
maze1TM

{(0, 0): {'right': (0, 1), 'down': (1, 0)},
 (0, 1): {'left': (0, 0), 'right': (0, 2)},
 (0, 2): {'left': (0, 1), 'right': (0, 3)},
 (0, 3): {'left': (0, 2), 'down': (1, 3)},
 (0, 4): {},
 (0, 5): {'right': (0, 6), 'down': (1, 5)},
 (0, 6): {'left': (0, 5), 'right': (0, 7), 'down': (1, 6)},
 (0, 7): {'left': (0, 6), 'right': (0, 8)},
 (0, 8): {'left': (0, 7), 'right': (0, 9), 'down': (1, 8)},
 (0, 9): {'left': (0, 8), 'down': (1, 9)},
 (1, 0): {'up': (0, 0), 'down': (2, 0)},
 (1, 1): {},
 (1, 2): {},
 (1, 3): {'up': (0, 3), 'down': (2, 3)},
 (1, 4): {},
 (1, 5): {'up': (0, 5), 'right': (1, 6), 'down': (2, 5)},
 (1, 6): {'left': (1, 5), 'up': (0, 6), 'down': (2, 6)},
 (1, 7): {},
 (1, 8): {'up': (0, 8), 'right': (1, 9)},
 (1, 9): {'left': (1, 8), 'up': (0, 9), 'down': (2, 9)},
 (2, 0): {'up': (1, 0), 'right': (2, 1), 'down': (3, 0)},
 (2, 1): {'left': (2, 0), 'right': (2, 2), 'down': (3, 1)},
 (2, 2): {'left': (2, 1), 'right': (2, 3), 'down': (3, 2)},
 (2, 3): {'left': (2, 2), 'up': (1,

In [6]:
mazeAvalActs

{(0, 0): ['right', 'down'],
 (0, 1): ['left', 'right'],
 (0, 2): ['left', 'right'],
 (0, 3): ['left', 'down'],
 (0, 4): [],
 (0, 5): ['right', 'down'],
 (0, 6): ['left', 'right', 'down'],
 (0, 7): ['left', 'right'],
 (0, 8): ['left', 'right', 'down'],
 (0, 9): ['left', 'down'],
 (1, 0): ['up', 'down'],
 (1, 1): [],
 (1, 2): [],
 (1, 3): ['up', 'down'],
 (1, 4): [],
 (1, 5): ['up', 'right', 'down'],
 (1, 6): ['left', 'up', 'down'],
 (1, 7): [],
 (1, 8): ['up', 'right'],
 (1, 9): ['left', 'up', 'down'],
 (2, 0): ['up', 'right', 'down'],
 (2, 1): ['left', 'right', 'down'],
 (2, 2): ['left', 'right', 'down'],
 (2, 3): ['left', 'up', 'right'],
 (2, 4): ['left', 'right'],
 (2, 5): ['left', 'up', 'right', 'down'],
 (2, 6): ['left', 'up', 'right'],
 (2, 7): ['left', 'down'],
 (2, 8): [],
 (2, 9): ['up', 'down'],
 (3, 0): ['up', 'right', 'down'],
 (3, 1): ['left', 'up', 'right'],
 (3, 2): ['left', 'up'],
 (3, 3): [],
 (3, 4): [],
 (3, 5): ['up', 'down'],
 (3, 6): [],
 (3, 7): ['up', 'right'],
 

In [7]:
net_maze = Network(heading="Lab 4 Maze",
                bgcolor ="#242020",
                font_color = "white",
                height = "750px",
                width = "100%" 
)
nodeColors={
    "wall":"Blue",
    "path": "White",
    "Goal": "Green",
    "Start": "Red",
    "Enemy": "Orange",
    "Food": "Yellow"
}
nodeColorsList=[]

for node in mazeWorldGraph.origin.keys():
    if mainMaze[node[0],node[1]]==1:
        nodeColorsList.append(nodeColors["path"])
    elif mainMaze[node[0],node[1]]==0:
        nodeColorsList.append(nodeColors["wall"])
    elif mainMaze[node[0],node[1]]==3:
        nodeColorsList.append(nodeColors["Enemy"])
    else:
        nodeColorsList.append(nodeColors["Food"])

nodes=["-".join(str(item) for item in el) for el in mazeWorldGraph.origin.keys()]

x_coords = []
y_coords = []

for node in mazeWorldGraph.origin.keys():
    x,y=mazeWorldGraph.getLocation(node)
    x_coords.append(x)
    y_coords.append(y)

sizes=[10]*len(nodes)
net_maze.add_nodes(nodes, color=nodeColorsList, x=x_coords, y=y_coords, size=sizes, title=nodes)

for node in net_maze.nodes:
    node['label']=''

edge_weights = {(intTupleTostr(k), intTupleTostr(v2)) : k2 for k, v in mazeWorldGraph.origin.items() for k2, v2 in v.items()}

edges=[]

for node_source in mazeWorldGraph.nodes():
    for node_target, action in mazeWorldGraph.get(node_source).items():
        #node_target or node_source is a tuple -> convert to str
        if (intTupleTostr(node_source),intTupleTostr(node_target)) not in edges and (intTupleTostr(node_target), intTupleTostr(node_source)):
            net_maze.add_edge(intTupleTostr(node_source),intTupleTostr(node_target), label=edge_weights[(intTupleTostr(node_source),intTupleTostr(node_target))])
            edges.append((intTupleTostr(node_source),intTupleTostr(node_target)))

for node in net_maze.nodes:
    if node['id']==intTupleTostr(goalState):
        node['color']=nodeColors['Goal']
    elif node['id']==intTupleTostr(initState):
        node['color']=nodeColors['Start']

In [8]:
net_maze.toggle_physics(False)

In [9]:
mp1=MazeProblem(initState, goalState, mazeWorldGraph)

In [10]:
AstarAgent=ProblemSolvingMazeAgentAstar(initState, mazeWorldGraph, goals)
IDA_starAgent=IDA_StarSearchAgent(initState, mazeWorldGraph, goals)

In [11]:
maze_Env1=Lab5NavEnvironment(mazeWorldGraph, mainMaze)
#maze_Env2=Lab5NavEnvironment(mazeWorldGraph, mainMaze)

#maze_Env1=MazeNavigationEnvironment(mazeWorldGraph)
#maze_Env2=MazeNavigationEnvironment(mazeWorldGraph)

In [12]:
maze_Env1.add_thing(AstarAgent)

Hi
[(9.22, <Node (1, 8)>)]
The node (1, 8) is extracted from frontier:
The child node <Node (0, 8)>.
The child node <Node (1, 9)>.
[(9.899, <Node (1, 9)>), (12.0, <Node (0, 8)>)]
The node (1, 9) is extracted from frontier:
The child node <Node (0, 9)>.
The child node <Node (2, 9)>.
[(10.22, <Node (2, 9)>), (12.63, <Node (0, 9)>), (12.0, <Node (0, 8)>)]
The node (2, 9) is extracted from frontier:
The child node <Node (3, 9)>.
[(10.602, <Node (3, 9)>), (12.63, <Node (0, 9)>), (12.0, <Node (0, 8)>)]
The node (3, 9) is extracted from frontier:
The child node <Node (3, 8)>.
The child node <Node (4, 9)>.
[(9.809999999999999, <Node (3, 8)>), (11.062, <Node (4, 9)>), (12.0, <Node (0, 8)>), (12.63, <Node (0, 9)>)]
The node (3, 8) is extracted from frontier:
The child node <Node (3, 7)>.
The child node <Node (4, 8)>.
[(9.071, <Node (3, 7)>), (10.211, <Node (4, 8)>), (12.0, <Node (0, 8)>), (12.63, <Node (0, 9)>), (11.062, <Node (4, 9)>)]
The node (3, 7) is extracted from frontier:
The child node 

In [18]:
maze_Env1.run()

step 1:
Agent decided to do down.
Agent in (8, 2) with performance = 98.2
step 2:
Agent decided to do up.
Agent in (7, 2) with performance = 97.2
step 3:
Agent decided to do left.
Agent in (7, 1) with performance = 96.2
Agent in (7, 1) is under attack!
step 4:
Agent decided to do left.
Agent in (7, 0) with performance = 86.58
step 5:
Agent decided to do left.
Agent in (7, -1) with performance = 85.58
Agent in (7, -1) is under attack!
step 6:
Agent decided to do down.
Agent in (8, -1) with performance = 77.022
Agent in (8, -1) is under attack!
step 7:
Agent decided to do down.
Agent in (9, -1) with performance = 69.3198
Agent in (9, -1) is under attack!
step 8:
Agent decided to do left.
Agent in (9, -2) with performance = 62.387820000000005
Agent in (9, -2) is under attack!
step 9:
Agent decided to do left.
Agent in (9, -3) with performance = 56.149038000000004
step 10:
Agent decided to do left.
Agent in (9, -4) with performance = 55.149038000000004


In [14]:
#maze_Env2.add_thing(IDA_starAgent)

In [15]:
#maze_Env2.run()

In [16]:
net_maze.show("graphMaze1.html", notebook=False)

graphMaze1.html
